# Atelier sur la génétique statistique et l'épidémiologie génétique STAGE-Québec
## Thème 2 - Phénotypes moléculaires en épidémiologie génétique

Par Marc-André Legault (Université de Montréal) et Qihuang Zhang (Université McGill)

**31 juillet - 1er août 2025**

### Introduction

Dans cette section de l'atelier, nous allons comparer les résultats de deux approches complémentaires pour l'analyse TWAS (Transcriptome-Wide Association Study) : **S-PrediXcan** et **FUSION**. Cette comparaison méthodologique vous permettra de mieux comprendre les différences pratiques entre ces outils largement utilisés dans la communauté de recherche.

#### Contexte méthodologique
- **S-PrediXcan** utilise un modèle Elastic Net avec calibration pour la polygénicité
- **FUSION** propose plusieurs modèles de prédiction, incluant également l'Elastic Net

Pour cette analyse comparative, nous avons délibérément utilisé les modèles GTEx Pancreas dans les deux logiciels afin d'assurer une comparaison équitable et rigoureuse. Cette approche nous permettra d'évaluer la concordance entre les méthodes tout en identifiant leurs spécificités respectives.

#### Objectifs d'apprentissage
À la fin de cette section, vous serez en mesure de :
1. Interpréter les différences entre les résultats TWAS obtenus par différentes méthodes
2. Comprendre l'impact de la calibration pour la polygénicité
3. Évaluer la concordance entre les approches méthodologiques
4. Identifier les forces et limites de chaque méthode

Les fichiers de résultats sont disponibles dans les points de contrôle et nous supposerons qu'ils sont localisés comme suit :

In [ ]:
library(ggplot2)

### Chargement et préparation des données

Nous allons maintenant charger les résultats des deux analyses TWAS dans R et les fusionner pour permettre une comparaison directe. Cette étape est cruciale car elle nous permet d'examiner les mêmes gènes analysés par les deux méthodes.

**Points clés à retenir :**
- Les résultats S-PrediXcan incluent à la fois les statistiques calibrées et non calibrées
- Nous conserverons les deux versions pour illustrer l'effet de la calibration pour la polygénicité
- Les données seront fusionnées sur l'identifiant Ensembl des gènes (ENSG) pour assurer la correspondance exacte

In [ ]:
df_predixcan <- read.csv("results/twas_en_Pancreas.csv")

# Nous conservons les statistiques TWAS non calibrées dans un DF séparé.
df_predixcan_uncal <- df_predixcan[, c("gene", "gene_name", "uncalibrated_zscore", "uncalibrated_pvalue", "pred_perf_r2")]
names(df_predixcan_uncal) <- c("ENSG", "gene_label", "Z_PREDIXCAN_NO_CALIB", "P_PREDIXCAN_NO_CALIB", "R2_PREDIXCAN")

df_predixcan <- df_predixcan[, c("gene", "gene_name", "zscore", "pvalue", "pred_perf_r2")]
names(df_predixcan) <- c("ENSG", "gene_label", "Z_PREDIXCAN", "P_PREDIXCAN", "R2_PREDIXCAN")
nrow(df_predixcan)

In [ ]:
df_fusion <- read.csv(
    "results/fusion_GTEx_pancreas.tsv",
    sep="\t",
    stringsAsFactors=F
)
# Gardons seulement le modèle "enet" qui est la même approche de régression pénalisée
# que pour S-PrediXcan afin de maximiser la comparabilité.
df_fusion <- df_fusion[df_fusion$MODEL == "enet", ]
df_fusion <- df_fusion[, c("ID", "TWAS.Z", "TWAS.P", "MODELCV.R2")]
df_fusion <- df_fusion[complete.cases(df_fusion), ]
names(df_fusion) <- c("ENSG", "Z_FUSION", "P_FUSION", "R2_FUSION")
df_fusion$P_FUSION <- as.numeric(df_fusion$P_FUSION)
df_fusion$Z_FUSION <- as.numeric(df_fusion$Z_FUSION)

nrow(df_fusion)

In [ ]:
merged_uncalibrated <- merge(df_predixcan_uncal, df_fusion, by="ENSG")
merged <- merge(df_predixcan, df_fusion, by="ENSG")
nrow(merged)

### Comparaison des résultats non calibrés : FUSION vs S-PrediXcan

Commençons par comparer les valeurs P obtenues avec FUSION et S-PrediXcan **avant** la calibration pour la polygénicité. Cette première comparaison nous permettra d'observer la concordance brute entre les deux méthodes.

#### Seuil de significativité
Nous utiliserons un seuil de significativité ajusté selon la correction de Bonferroni : **P ≤ 0,05 / 1058**, où 1058 représente le nombre total de gènes disponibles dans les résultats des deux algorithmes.

**Interprétation du graphique :**
- **Axe X :** -log₁₀(valeur P) pour FUSION
- **Axe Y :** -log₁₀(valeur P) pour S-PrediXcan (non calibré)
- **Ligne diagonale rouge :** Concordance parfaite entre les méthodes
- **Lignes pointillées :** Seuil de significativité statistique

In [ ]:
p_thresh <- -log10(0.05 / 1058)

ggplot(merged_uncalibrated, aes(x = -log10(P_FUSION), y = -log10(P_PREDIXCAN_NO_CALIB))) +
    geom_rect(aes(xmin = -Inf, xmax = p_thresh, ymin = p_thresh, ymax = Inf), 
              fill = "#eff5ff", inherit.aes = FALSE) +
    geom_rect(aes(xmin = p_thresh, xmax = Inf, ymin = p_thresh, ymax = Inf), 
              fill = "#ffeff5", inherit.aes = FALSE) +
    geom_rect(aes(xmin = p_thresh, xmax = Inf, ymin = -Inf, ymax = p_thresh), 
              fill = "#f5ffef", inherit.aes = FALSE) +
    geom_point() +
    geom_vline(xintercept = -log10(0.05 / 1058), linetype = "dotted", color = "#888888", size = 1) +
    geom_hline(yintercept = -log10(0.05 / 1058), linetype = "dotted", color = "#888888", size = 1) +
    geom_abline(intercept = 0, slope = 1, linetype = "dashed", color = "red") +
    theme_bw()

<div class="alert alert-info">
    <h4>Questions d'analyse critique</h4>
    
    <p><strong>Interprétation des régions colorées :</strong></p>
    <p>Trois régions distinctes sont identifiées dans le graphique à l'aide de couleurs différentes. Analysez chacune de ces régions :</p>
    <ul>
        <li><strong>Région bleue :</strong> Que représentent les gènes dans cette zone ? Quelle méthode les identifie comme significatifs ?</li>
        <li><strong>Région rose :</strong> Comment interpréter les gènes situés dans cette région ?</li>
        <li><strong>Région verte :</strong> Qu'indique cette zone sur la performance comparative des méthodes ?</li>
    </ul>
    
    <p><strong>Exercice pratique :</strong></p>
    <p>En utilisant les données disponibles, identifiez des exemples concrets de gènes qui sont :</p>
    <ul>
        <li><strong>Significatifs avec les deux approches</strong> (concordance méthodologique)</li>
        <li><strong>Significatifs uniquement avec FUSION</strong> (spécificité de la méthode)</li>
        <li><strong>Significatifs uniquement avec S-PrediXcan</strong> (spécificité de la méthode)</li>
    </ul>
    
    <p><strong>Réflexion méthodologique :</strong></p>
    <p>Quelles conclusions peut-on tirer sur la concordance entre les différentes méthodes TWAS ? Ces différences sont-elles préoccupantes ou attendues ?</p>
</div>

In [ ]:
# Vous pouvez utiliser cette cellule pour explorer les données !

### Comparaison avec les résultats calibrés : Impact de l'ajustement pour la polygénicité

Nous répétons maintenant la même analyse comparative en utilisant les statistiques de S-PrediXcan qui ont été **calibrées pour la polygénicité**. Cette calibration est une caractéristique distinctive de S-PrediXcan qui vise à corriger l'inflation des statistiques due à la structure de déséquilibre de liaison.

#### Concept clé : La calibration pour la polygénicité

La polygénicité (influence de multiples variants génétiques sur un trait) peut entraîner une inflation des statistiques TWAS. S-PrediXcan propose une méthode de calibration qui :
- Ajuste les statistiques Z en tenant compte de la corrélation entre les prédicteurs génétiques
- Réduit les faux positifs dus à la structure de déséquilibre de liaison
- Fournit des valeurs P plus conservatrices et potentiellement plus fiables

#### Questions de discussion

<div class="alert alert-info">
    <h4>Analyse comparative approfondie</h4>
    
    <p><strong>Impact de la calibration :</strong></p>
    <ul>
        <li>Comment l'ajustement pour la polygénicité influence-t-il les valeurs P de S-PrediXcan ?</li>
        <li>Observe-t-on un déplacement systématique des points par rapport au graphique précédent ?</li>
        <li>Les résultats deviennent-ils plus ou moins significatifs après calibration ?</li>
    </ul>
    
    <p><strong>Évaluation méthodologique :</strong></p>
    <ul>
        <li>À quelle approche devrait-on accorder plus de confiance : les résultats calibrés ou non calibrés ?</li>
        <li>Comment cette calibration affecte-t-elle la concordance entre FUSION et S-PrediXcan ?</li>
        <li>Quelles sont les implications pratiques pour l'interprétation des résultats TWAS ?</li>
    </ul>
    
    <p><strong>Considérations pour la recherche :</strong></p>
    <ul>
        <li>Dans quels contextes serait-il préférable d'utiliser FUSION par rapport à S-PrediXcan ?</li>
        <li>Comment ces différences méthodologiques influencent-elles les conclusions biologiques ?</li>
    </ul>
</div>

**Note pratique :** Il est recommandé de comparer attentivement les deux graphiques pour identifier les gènes dont le statut de significativité change après calibration. Ces cas sont particulièrement informatifs pour comprendre l'impact de la correction pour la polygénicité.

In [ ]:
p_thresh <- -log10(0.05 / 1058)

ggplot(merged, aes(x = -log10(P_FUSION), y = -log10(P_PREDIXCAN))) +
    geom_rect(aes(xmin = -Inf, xmax = p_thresh, ymin = p_thresh, ymax = Inf), 
              fill = "#eff5ff", inherit.aes = FALSE) +
    geom_rect(aes(xmin = p_thresh, xmax = Inf, ymin = p_thresh, ymax = Inf), 
              fill = "#ffeff5", inherit.aes = FALSE) +
    geom_rect(aes(xmin = p_thresh, xmax = Inf, ymin = -Inf, ymax = p_thresh), 
              fill = "#f5ffef", inherit.aes = FALSE) +
    geom_point() +
    geom_vline(xintercept = p_thresh, linetype = "dotted", color = "#888888", size = 1) +
    geom_hline(yintercept = p_thresh, linetype = "dotted", color = "#888888", size = 1) +
    geom_abline(intercept = 0, slope = 1, linetype = "dashed", color = "red") +
    theme_bw()